# Genshin Impact Research

I obtained a research paper PDF from https://www.researchgate.net/publication/373922639_Research_on_the_Reasons_for_the_Success_of_Genshin_Impact.

Available at assets/genshin_impact.pdf

In [1]:
!pip install weaviate-client langchain langchain-text-splitters langchain_weaviate langchain-huggingface langchain-community tiktoken pypdf rapidocr-onnxruntime sentence-transformers python-dotenv ipywidgets

In [2]:
!jupyter labextension enable widgetsnbextension
from ipywidgets import FloatProgress

In [3]:
%load_ext dotenv
%dotenv

In [4]:
import os
import langchain_community
import langchain_huggingface
import langchain_text_splitters
import weaviate

In [5]:
client = weaviate.connect_to_weaviate_cloud(cluster_url=os.environ["WEAVIATE_API_URL"], auth_credentials=weaviate.auth.AuthApiKey(os.environ["WEAVIATE_API_KEY"]))

In [6]:
embeddings = langchain_huggingface.HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cuda"})

/root/projects/langchain_stuff/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/root/projects/langchain_stuff/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
from langchain_community.document_loaders import PyPDFLoader
loader = langchain_community.document_loaders.PyPDFLoader("assets/genshin_impact.pdf", extract_images=True)
document = loader.load()

In [8]:
splitter = langchain_text_splitters.RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

In [9]:
chunks = splitter.split_documents(document)

In [10]:
chunks[:10]

[Document(metadata={'source': 'assets/genshin_impact.pdf', 'page': 0}, page_content='Research on the Reasons for the Success of Genshin Impact   \nRan Ren1,a,* \n1Vanke Meisha Academy, Shenzhen, Guangdong, 518000, China \na. renran@stu.vma.edu.cn \n*corresponding author  \nAbstract:  Genshin Impact has taken the gaming world by storm. Developed and published by \nmiHoYo, this game has captured the attention of gamers worldwide with its stunning visuals, \ninteresting storyline, and engaging gameplay. Genshin Impact is an open- world game that \nallows players to explore a vast world and interact with a variety of characters. The game also'),
 Document(metadata={'source': 'assets/genshin_impact.pdf', 'page': 0}, page_content="has very diverse mechanics, including combat, adventure, puzzle solving, character development, and more. Players need to constantly upgrade their character level, combat ability, and equipment level in order to better explore and complete quests in the game. This 

In [11]:
import langchain_weaviate.vectorstores

In [12]:
vectordb = langchain_weaviate.vectorstores.WeaviateVectorStore.from_documents(chunks, embeddings, client=client, by_text=False)

In [13]:
for page in vectordb.similarity_search("What is Genshin Impact?", k=3):
    print(page.page_content)

[2] Chinese Genshin Impact is one of the most -profitable video games on Earth. It balances mass appeal with Beijing’s 
blessing. (2022, June). Retrieved March 8, 2023, from The World from PRX website: 
https://theworld.org/stories/2022- 06-21/chinese -genshin -impact -one-most -profitable -video -games -earth -it-
balances -mass 
[3] Greting, M., Mao, X., & Eladhari, M. P. (2022). What Inspires Retellings -  A Study of the Game Genshin Impact. 
Interactive Storytelling, 249 –269. https://doi.org/10.1007/978- 3-031-22298- 6_16  
[4] Jones, R. (2021, August 25). The best Genshin Impact Xiangling build. Rock Paper Shotgun; Rock  Paper Shotgun. 
https://www.rockpapershotgun.com/best -genshin- impact -xiangling -build  
[5] Jiang, S. (2020, November 3). 11 Real Chinese Foods In Genshin Impact. Retrieved March 8, 2023, from Kotaku 
website: https://kotaku.com/11 -real-chinese- foods -in-genshin- impact-1845559337
This paper will explore the world of Genshin Impact, using its scenario design

In [14]:
for page in vectordb.similarity_search("Why is Genshin Impact so successful?", k=3):
    print(page.page_content)

This paper will explore the world of Genshin Impact, using its scenario design, character design and its analysis of cultural export to examine why Genshin Impact is so popular. The rich character setting and storyline, unique Chinese elements, exquisite graphics and sound effects, and Genshin Impact's aggressive cultural exportation make this game a favorite and sought -after game by players. The point 
of this study is to get a deeper understanding of this game and to understand what mechanics are behind the game's success, to analyze the parts of the game that are well designed, and to understand the public's preferences for the game [3] . Proceedings of the International Conference on Social Psychology and Humanity Studies
DOI: 10.54254/2753-7064/5/20230281
© 2023 The Authors. This is an open access article distributed under the terms of the Creative Commons Attribution License 4.0
(https://creativecommons.org/licenses/by/4.0/).
285
2.3. Scene Music  
In addition to the visuals and

In [15]:
import langchain_core

In [16]:
prompt = langchain_core.prompts.ChatPromptTemplate.from_template("""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
""")

In [17]:
model = llm=langchain_huggingface.HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.1", temperature=1, model_kwargs={"huggingface_api_token": os.environ["HUGGINGFACE_API_KEY"], "max_length": 180})

In [18]:
output_parser = langchain_core.output_parsers.StrOutputParser()

In [19]:
retriever = vectordb.as_retriever()

In [20]:
retrieval_chain = {"context": retriever,  "question": langchain_core.runnables.RunnablePassthrough()} | prompt | model | output_parser

In [21]:
print(retrieval_chain.invoke("What is Genshin Impact?"))

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 (Request ID: lPj2QP79pcceQnJL-kc5m)

Rate limit reached. Please log in or use a HF access token

In [ ]:
print(retrieval_chain.invoke("Why is Genshin Impact so successful?"))